In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/personal_finance_employees_V1.csv')
df.head()

,Employee,Monthly Income (£),Electricity Bill (£),Gas Bill (£),Netflix (£),Amazon Prime (£),Groceries (£),Transportation (£),Water Bill (£),Sky Sports (£),Other Expenses (£),Savings for Property (£),Monthly Outing (£)
0,Employee_1,4999.39,120.000000,80.0000,12,4.99,239.69000,149.11000,40.0,70.0,100.000000,300.0,120.0
1,Employee_2,4660.24,120.000000,80.0000,12,4.99,226.24000,197.23000,NaN,70.0,100.000000,300.0,120.0
2,Employee_3,1942.89,124.679456,98.1346,12,4.99,333.77745,201.27956,NaN,70.0,125.770693,NaN,120.0
3,Employee_4,3565.80,120.000000,80.0000,12,4.99,213.04000,154.67000,NaN,NaN,100.000000,300.0,120.0
4,Employee_5,5121.99,120.000000,80.0000,12,4.99,182.56000,190.71000,NaN,70.0,100.000000,300.0,NaN
